In [ ]:
bfce_test = Sequential()
Xception = keras.applications.Xception(include_top=False, weights="imagenet")
for layer in Xception.layers:
    layer.trainable=False
# for layer in Xception.layers[-4:]:
#     layer.trainable=True
bfce_test.add(Xception)
bfce_test.add(Flatten())
bfce_test.add(Dense(512, activation = "relu"))
bfce_test.add(BatchNormalization())
bfce_test.add(Dropout(.3))

bfce_test.add(Dense(512, activation = "relu"))
bfce_test.add(BatchNormalization())
bfce_test.add(Dropout(.2))
    
bfce_test.add(Dense(512, activation = "relu"))
bfce_test.add(BatchNormalization())
bfce_test.add(Dropout(.2))

bfce_test.add(Dense(512, activation = "relu"))
bfce_test.add(BatchNormalization())
bfce_test.add(Dropout(.2))



bfce_test.add(Dense(37, activation='softmax'))

bfce_test.compile(
    optimizer = Adam(),
    loss = CategoricalCrossentropy(),
    metrics = [keras.metrics.F1Score(average='weighted'), "precision", "recall"]
)


bfce_test.fit(train_generator, 
             epochs=1,
             class_weight = class_weights
    
)

In [ ]:
test = Sequential()
Xception = keras.applications.Xception(include_top=False, weights="imagenet")
for layer in Xception.layers:
    layer.trainable=False
# for layer in Xception.layers[-4:]:
#     layer.trainable=True
test.add(Xception)
test.add(Flatten())
test.add(Dense(512, activation = "relu"))
test.add(BatchNormalization())
test.add(Dropout(.5))

test.add(Dense(512, activation = "relu"))
test.add(BatchNormalization())
test.add(Dropout(.2))
    
test.add(Dense(512, activation = "relu"))
test.add(BatchNormalization())
test.add(Dropout(.2))

test.add(Dense(512, activation = "relu"))
test.add(BatchNormalization())
test.add(Dropout(.2))



test.add(Dense(37, activation='sigmoid'))


test.compile(
    optimizer = Adam(),
    loss = BinaryCrossentropy(),
    metrics = [keras.metrics.F1Score(average='weighted'), "accuracy", "precision", "recall"]
)


test.fit(train_generator, 
             steps_per_epoch=93307//batch_size, 
             epochs=1,
             class_weight = class_weights
    
)

In [ ]:
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))

##Can't use adagrad for some reason, it won't run

In [ ]:
###AGGREGATE PREDICTIONS ACROSS ALL SCREENSHOTS OF A VIDEOGAME 

class AggregatePredictions(Callback):
    def __init__(self, game_names):
        super().__init__()
        self.game_names = game_names
        self.aggregated_predictions = defaultdict(list)

    def on_epoch_end(self, epoch, logs=None):
        # Iterate over each video-game
        for game in self.game_names:
            # Initialize an empty list to store predictions for this game
            game_predictions = []
            # Aggregate predictions across batches
            for batch_predictions in self.aggregated_predictions[game]:
                game_predictions.extend(batch_predictions)
            # Take the mean prediction across all screenshots for each tag
            avg_predictions = np.mean(game_predictions, axis=0)
            # Store the aggregated predictions for this video-game
            self.aggregated_predictions[game] = [avg_predictions]

    def on_batch_end(self, batch, logs=None):
        # Retrieve predictions for each video-game from the current batch
        batch_game_names = logs.get('appid', [])
        batch_predictions = logs.get('predictions', [])
        # Update aggregated predictions for each video-game
        for game, predictions in zip(batch_game_names, batch_predictions):
            self.aggregated_predictions[game].append(predictions)

# Example usage:
# Define your model and compile it
# model = ...
# model.compile(...)

# Instantiate the AggregatePredictions callback
# aggregate_predictions_callback = AggregatePredictions(game_names=['game1', 'game2'])

# Train your model using the callback
# model.fit(..., callbacks=[aggregate_predictions_callback])




# You can pass a list of callbacks (as the keyword argument callbacks) to the following model methods:

# keras.Model.fit()
# keras.Model.evaluate()
# keras.Model.predict()

In [ ]:
import tensorflow as tf
from sklearn.metrics import f1_score

# Define a custom metric class for F1 score
class CustomF1Score(tf.keras.metrics.Metric):
    def __init__(self, name='custom_f1_score', **kwargs):
        super(CustomF1Score, self).__init__(name=name, **kwargs)
        self.f1_score = self.add_weight(name='f1_score', initializer='zeros')
        
    def update_state(self, y_true, y_pred_prob, **kwargs):
        # Calculate the number of true labels per instance
        num_true_labels = tf.reduce_sum(tf.cast(y_true > 0, tf.float32), axis=1, keepdims=True)
        
        # Calculate the threshold dynamically
        threshold = 0.2 / num_true_labels

        # Apply the threshold to obtain binary predictions
        y_pred = tf.cast(y_pred_prob > threshold, tf.float32)
        
        # Calculate the F1 score
        f1 = f1_score(y_true, y_pred, average='weighted')
        
        # Update the state of the F1 score metric
        self.f1_score.assign(f1)

    def result(self):
        return self.f1_score

# Create an instance of the custom F1 score metric
custom_f1_metric = CustomF1Score()

# Update the state of the custom metric using the validation data
custom_f1_metric.update_state(y_true, y_pred_prob)

# Print the F1 score
print("Custom F1 Score:", custom_f1_metric.result().numpy())

In [ ]:
###REPORT FOR CCE

from sklearn.metrics import classification_report


# Get the predicted probabilities for the validation/test set
#valid_generator.reset()
#y_pred_prob = bfce_test.predict(valid_generator)

# Get the ground truth labels for the validation/test set
y_true = valid_generator.labels

num_true_labels = np.sum(y_true > 0, axis=1) 
num_true_labels = num_true_labels[:, np.newaxis]

y_true = valid_generator.labels*num_true_labels

# Threshold the predicted probabilities to obtain binary predictions
y_pred = (y_pred_prob > 0.2/num_true_labels).astype(int)

# Calculate classification report
report = classification_report(y_true, y_pred, target_names=tags_to_use)

print(report)

In [ ]:
###LOADING IMAGES IN BATCHES
###This whole step os sp that the images are only loaded into memory and used for training in batches
# Define custom image data generator so we can random crop and resize img in preprocessing
##
# class CustomImageDataGenerator(ImageDataGenerator):
#     def __init__(self, *args, crop_size, target_size = None, **kwargs):
#         super().__init__(*args, **kwargs)
#         if target_size is None:
#             target_size=(299, 299)
#         self.target_size = target_size
#         self.crop_size = crop_size

#     def img_random_crop(self, img):
#         h, w = img.shape[:2]
#         new_h, new_w = self.crop_size[:2]
        
#         # Calculate random cropping offsets
#         top = np.random.randint(0, h - new_h)
#         left = np.random.randint(0, w - new_w)

#         # Perform random crop
#         img = img[top:top + new_h, left:left + new_w]

#         return img
    
#     def img_resize(self, img):
#         img = resize(img, self.target_size, anti_aliasing=True)
#         return img
    
#     def flow_from_dataframe(self, *args, **kwargs):
#         for batch_x, batch_y in super().flow_from_dataframe(*args, **kwargs):        
#             try:
#                 batch_x = np.array([self.img_random_crop(img) for img in batch_x])
#                 batch_x = np.array([self.img_resize(img) for img in batch_x])
#                 batch_x = keras.applications.xception.preprocess_input(batch_x) #Xception model requires this, normalizing the pixel values
#                 yield (batch_x, batch_y)
#             except Exception as e:
#                 # Print the file paths of the images causing the error
#                 print("Error processing batch. File paths:")
#                 print(batch_x)
                
#                 # Optionally, print the specific error message
#                 print("Error message:", e)
                
#                 continue
            
# print(id(CustomImageDataGenerator))

# # Define image data generator with data augmentation and random crop
# datagen = CustomImageDataGenerator(
#     #horizontal_flip=True,
#     #fill_mode='nearest',
#     crop_size=(897,897))

In [ ]:
from keras.callbacks import History

history = History()
print(bfce_test.history)

In [ ]:


def preprocess_and_save_images(DF, source_dir, save_dir, target_size=(299, 299), crop_size=897):
    os.makedirs(save_dir, exist_ok=True)
    for row in tqdm(Train_DF.iterrows()):
        # Read image
        img_path = os.path.join(source_dir, row['screenshots'])
        img = cv2.imread(img_path)
        if img is None:
            continue  # Skip if unable to read image
        # # # Random crop
        # h, w = img.shape[:2]
        # crop_size = min(crop_size, h, w)
        # # Calculate random cropping offsets
        # top = np.random.randint(0, h - crop_size + 1)
        # left = np.random.randint(0, w - crop_size + 1)
        # Perform crop
        # img = img[top:top + crop_size, left:left + crop_size]

        # Resize
        img = resize(img, target_size, anti_aliasing=True, preserve_range=True)
        

        # Save preprocessed image with original filename
        filename = os.path.basename(img_path)
        save_path = os.path.join(save_dir, filename)
        cv2.imwrite(save_path, img)
    

source_directory = '/Users/sammcmanagan/AA Assignment 2/images'
save_directory = '/Users/sammcmanagan/AA Assignment 2/resized_images'



###30,161 images in the train df that aren't 1920x1080

In [ ]:
###CHECKING ISSUES WITH LOADING SOME OF THE IMAGES
import os
from PIL import Image

def check_images(directory):
    error_count = 0
    total_images = 0
    
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            total_images += 1
            try:
                with Image.open(filepath) as img:
                    # Do nothing, just open the image to check for errors
                    pass
            except Exception as e:
                print(f"Error opening image {filename}: {e}")
                error_count += 1
                
    print(f"Total images checked: {total_images}")
    print(f"Errors encountered: {error_count}")

# Example usage:
directory_path = "/Users/sammcmanagan/AA Assignment 2/Images"
tqdm(check_images(directory_path))

##three images won't load in the directory, these will be deleted

In [ ]:

# def none_images():
#     Noimg = 0
#     imgs = 0
#     for _, row in tqdm(Valid_DF.iterrows()):
#         img_path = row['screenshots']
#         img = cv2.imread(img_path)
#         if img is None:  # Skip if unable to read image
#             Noimg += 1
#             continue
#         imgs+=1
#     print(Noimg)
#     print(imgs)
# none_images()

In [ ]:
# import os

# def get_files_not_in_resized(original_dir, resized_dir):
#     original_files = set(os.listdir(original_dir))
#     resized_files = set(os.listdir(resized_dir))
    
#     files_not_in_resized = original_files - resized_files
    
#     return files_not_in_resized

# # Example usage
# original_dir = '/Users/sammcmanagan/AA Assignment 2/Images'
# resized_dir = '/Users/sammcmanagan/AA Assignment 2/resized_images'

# files_not_in_resized = get_files_not_in_resized(original_dir, resized_dir)
# print("Files not present in resized_images directory:")
# for file_name in files_not_in_resized:
#     print(file_name)
# print(len(files_not_in_resized))
# print(len(Test_DF))

# directory='/Users/sammcmanagan/AA Assignment 2/Images'
# abs_file_names = []

# for file_name in Valid_DF['screenshots']:
#     tmp = os.path.abspath(directory+os.sep+file_name)
#     abs_file_names.append(tmp)

# # update dataframe
# Train_DF['screenshots'] = abs_file_names

# pd.set_option('display.max_colwidth', None)
# print(Train_DF['screenshots'].head(1))

In [ ]:
# file_path=Train_DF['screenshots']
# for screenshot in tqdm(Train_DF['screenshots']):
#     file_path=screenshot
#     try:
#         image=cv2.imread(file_path)
#         shape=image.shape
#     except:
#         print('Invalid image file')
#         print(screenshot)